In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import datetime
import numpy as np
import matplotlib.pyplot as plt
%load_ext tensorboard

2022-12-13 14:42:40.883969: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-13 14:42:41.313625: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/hendrik/anaconda3/envs/iannwtf-gpu/lib/
2022-12-13 14:42:41.313778: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/hendrik/anaconda3/envs/iannwtf-gpu/lib/
2022-12-13 14:42:41.313783: W tensor

In [2]:
# import dataset
(train_ds, test_ds), ds_info = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info=True)

2022-12-13 14:42:42.188677: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 14:42:42.192281: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 14:42:42.192388: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 14:42:42.192810: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [24]:
print(ds_info)

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='/home/hendrik/tensorflow_datasets/mnist/3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [64]:
def new_target_fnc(ds, sequence_len):
    """
    Creates list of new targets by alternately adding and subtracting
    The first digit is added, the second subtracted, the third added, etc
    Parameters
    ----------
    ds : TensorFlowDataset
    original mnist dataset containg images and targets as a tuple.
    sequence_len : int
    indicates at which point the sum has to reset for the new sequence
    Returns
    -------
    l : list
    list containing the new targets
    """
    l = list()
    for i, elem in enumerate(ds):
        if (i % sequence_len) == 0:
            l.append(int(elem[1]))
        else:
            if (i % 2) == 0:
                l.append(int(l[i-1] + elem[1]))
            else:
                l.append(int(l[i-1] - elem[1]))
    return l


# preprocess the data
def preprocess(mnist, window_size):

    # convert data from uint8 to float32
    mnist = mnist.map(lambda img, target: (tf.cast(img, tf.float32), target))
    # bringing image values from range [0,255] to [-1,1]
    mnist = mnist.map(lambda img, target: ((img/128.)-1., target))
    # generate new targets
    new_targets = new_target_fnc(mnist,window_size)
    # convert new targets into a dataset
    new_targets = tf.data.Dataset.from_tensor_slices(new_targets)
    # zip both datasets
    mnist = tf.data.Dataset.zip((mnist, new_targets))
    # replace old targets
    mnist = mnist.map(lambda img, target: (img[0], target))
    # generate multiple sequences
    mnist = mnist.window(window_size, drop_remainder=True)
    # cache progress in memory
    mnist = mnist.cache()
    # shuffle, batch, prefetch
    mnist = mnist.shuffle(1000)
    mnist = mnist.batch(32)
    mnist = mnist.prefetch(20)
    
    # return processed dataset
    return mnist

In [65]:
#train_data = preprocess(train_ds, 10)
test_data = preprocess(test_ds, 10)

before zip
<MapDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
after zip
<ZipDataset element_spec=((TensorSpec(shape=(28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None)), TensorSpec(shape=(), dtype=tf.int32, name=None))>
<MapDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>


In [66]:
#visualize a sample of the dataset
iterator = iter(train_data)
print(iterator.get_next())
#tf.shape(iterator.get_next())

(<tensorflow.python.data.ops.dataset_ops._NestedVariant object at 0x7f15e42c1730>, <tensorflow.python.data.ops.dataset_ops._NestedVariant object at 0x7f1573e0d070>)


2022-12-13 15:50:48.645403: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [46]:
'''
own implementation of a RNNCell
takes the current data and the hidden / cell state of the previous cell as input
'''
class AbstractLayer(tf.keras.layers.AbstractRNNCell):
    def __init__(self, unit_size):
        super().__init__()
              
        # since there are three gates we need three Dense layers
        # @TODO size should be the same as input, I guess
        self.forget_gate = tf.keras.layers.Dense(unit_size, activation="sigmoid")
        self.input_gate = tf.keras.layers.Dense(unit_size, activation="sigmoid")
        self.candidate_gate = tf.keras.layers.Dense(unit_size, activation="tanh")
        self.output_gate = tf.keras.layers.Dense(unit_size, activation="sigmoid")
        
    def call(self, data, states):
        
        self.hidden_state = states[0]
        self.cell_state = states[1]
        # concatinate the hidden state and the new input together, to have in one list
        concat_hidden_input = self.hidden_state.concat(data)
        
        # calculate the new cell state
        forget_value = self.forget_gate(concat_hidden_input)
        input_value = self.input_gate(concat_hidden_input)
        candidate_value = self.candidate_gate(concat_hidden_input)
        new_cell_state = forget_value * self.hidden_state + input_value * candidate_value
        
        # calculate the new hidden state
        output_value = self.output_gate(concat_hidden_input)
        new_hidden_state = output_value * tf.math.tanh(new_cell_state)
        
        # return the hidden state as an output of the current cell and a list containing the new states
        return new_hidden_state, list(new_hidden_state, new_cell_state)
    
    @property
    def state_size(self):
        return [tf.TensorShape([self.forget_gate]),
               tf.TensorShape([self.input_gate]),
               tf.TensorShape([self.candidate_gate]),
               tf.TensorShape([self.output_gate])]
    
    @property
    def output_size(self):
        return [tf.TensorShape([self.output_gate])]
    
    def get_initial_state(inputs=None, batch_size=None, dtype=None):
        return [tf.zeros([self.forget_gate]),
               tf.zeros([self.input_gate]),
               tf.zeros([self.candidate_gate]),
               tf.zeros([self.output_gate])]

In [47]:
class RNNModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
        #basic CNN structure to convert mnist matrix into a vector
        input_shape = (32,10,28,28,1)
        self.cnn_list = [tf.keras.layers.Conv2D(filters=24, kernel_size=3, padding='same', activation='relu', input_shape=input_shape[2:]),
                        tf.keras.layers.Conv2D(filters=24, kernel_size=3, padding='same', activation='relu', input_shape=input_shape[2:]),
                        tf.keras.layers.TimeDistributed(tf.keras.layers.GlobalAvgPool2D())]
        
        # get instance of our own implementation
        self.lstm_cell = AbstractLayer(10)
        # feed that instance into the wrapper class
        self.rnn_layer = tf.keras.layers.RNN(self.lstm_cell, return_sequences=False, unroll=True)
        # output layer
        self.output_layer = tf.keras.layers.Dense(1, activation="sigmoid")
        # metrics
        self.metrics_list = [tf.keras.metrics.Mean("loss"),
                            tf.keras.metrics.CategoricalAccuracy()]
       
    @property
    def metrics(self):
        return self.metrics_list
        
    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset_state()
            
    def call(self, sequence, training=False):
        
        # call CNN first
        rnn_output = sequence
        for layer in self.cnn_list:
            rnn_output = layer(rnn_output)
            
        # call RNN second
        rnn_output = self.rnn_layer(rnn_output)
        
        return self.output_layer(rnn_output)
    
    
    def train_step(self, data):
        """
        Standard train_step method, assuming we use model.compile(optimizer, loss, ...)
        """
        
        sequence, label = data
        with tf.GradientTape() as tape:
            output = self(sequence, training=True)
            loss = self.compiled_loss(label, output, regularization_losses=self.losses)
        gradients = tape.gradient(loss, self.trainable_variables)
        
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        self.metrics[0].update_state(loss)
        self.metrics[1].update_state(label, output)
        
        return {m.name : m.result() for m in self.metrics}
    
    def test_step(self, data):
        
        """
        Standard test_step method, assuming we use model.compile(optimizer, loss, ...)
        """
        
        sequence, label = data
        output = self(sequence, training=False)
        loss = self.compiled_loss(label, output, regularization_losses=self.losses)
                
        self.metrics[0].update_state(loss)
        self.metrics[1].update_state(label, output)
        
        return {m.name : m.result() for m in self.metrics}

In [48]:
model = RNNModel()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss = tf.keras.losses.CategoricalCrossentropy()

# compile the model (here, adding a loss function and an optimizer)
model.compile(optimizer = optimizer, loss=loss)   

In [49]:
EXPERIMENT_NAME = "RNN_noise"
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logging_callback = tf.keras.callbacks.TensorBoard(log_dir=f"./logs/{EXPERIMENT_NAME}/{current_time}")

In [50]:
history = model.fit(train_data,
                    validation_data=test_data,
                    initial_epoch=25,
                    epochs=50,
                    callbacks=[logging_callback])

Epoch 26/50


TypeError: in user code:

    File "/home/hendrik/anaconda3/envs/iannwtf-gpu/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/hendrik/anaconda3/envs/iannwtf-gpu/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/hendrik/anaconda3/envs/iannwtf-gpu/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/tmp/ipykernel_16404/972140754.py", line 49, in train_step
        output = self(sequence, training=True)
    File "/home/hendrik/anaconda3/envs/iannwtf-gpu/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filez_0f93gv.py", line 24, in tf__call
        ag__.for_stmt(a

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.legend(labels=["training","validation"])
plt.xlabel("Epoch")
plt.ylabel("Binary Crossentropy Loss")
plt.show()

Problem with homework07

Calling our Conv2D layers produced an error that we were not able to interpret correctly. We believe it to be due to our processing function. It might produce a wrong shape, but we were not able to figure out what was wrong with the shape, as we couldn't handle the _NestedVariant object that our function produced.